# Coverting Excel templates using nmparser API

In [ ]:
_URL_PARSER = "https://api.ideaconsult.net/gracious/enm/gracious/nmparser"
_URL_PARSER = "https://api.ideaconsult.net/nanoreg1/enm/nanoreg1/nmparser"


config = {
    "files[]": ["doseresponse/GRACIOUS-TDRF-invitro-WP5-P8-WST-1_nanomaterials.xlsx"],
    "jsonconfig": "doseresponse/WST1.json",
    "expandconfig": "./doseresponse/expand.json"
}

In [ ]:
import requests
import io


def call_nmparser(config, url=_URL_PARSER):
    form = {}
    for key, value in config.items():
        if isinstance(value, str):
            # If the value is a string (file path), convert it to a list
            value = [value]
        for file_path in value:
            with open(file_path, 'rb') as file:
                file_content = file.read()
                file_name = file.name
            form[key] = (file_name, io.BytesIO(file_content))

    try:
        print(form)
        response = requests.post(url, files=form)
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(response.status_code)
    except Exception as err:
        raise err

parsed_json = call_nmparser(config)

for key in parsed_json:
    print(key,parsed_json[key])


In [ ]:
# explore the json structure

from IPython.display import JSON

for substance in parsed_json["substance"]:
    print(substance["name"])
    for study in substance["study"]:
        for key in study:
            if key == "effects":
                for effect in study[key]:
                    print(effect)
            else:
                print(key,":",study[key])

    #JSON(substance["study"])

# Converting AMBIT Json to NeXus using REST API 
(under development)

In [ ]:
import json

def ambit2nexus(substance_json, url="https://api.ramanchada.ideaconsult.net/dataset/convert"):
    try:
        response = requests.post(url,json =substance_json)
        if response.status_code == 200:
            return response.json()
        else:
            raise(Exception(response.status_code))
    except Exception as err:
        raise err

task = ambit2nexus(parsed_json)
task

# Parsing JSON with pynanomapper 

In [ ]:
from  pynanomapper.datamodel.ambit import Substances

s = Substances(**parsed_json)
for substance in s.substance:

    print(substance.name)
    print(substance.i5uuid)
    for study in substance.study:
        print(study.owner)
        print(study.protocol)
        print(study.citation)
        print(study.parameters)
        for effect in study.effects:
            print(effect.endpoint,effect.endpointtype,{key: value for key, value in effect.result if value is not None})
            print(effect.conditions)
    print()

# Convert to NeXus with pynanomapper

In [ ]:
from  pynanomapper.datamodel.ambit import Substances
from  pynanomapper.datamodel.nexus_writer import to_nexus
import nexusformat.nexus.tree as nx

import os
_nxroot = nx.NXroot()
_substance = None
_substance = Substances(**parsed_json)
_substance.to_nexus(_nxroot)
_nxroot.save("substances.nxs", 'w')


In [ ]:
parsed_json
json_data =parsed_json["substance"][0]["study"][0]
parsed_json["substance"][0]

In [ ]:
from  pynanomapper.datamodel.ambit import Study
from  pynanomapper.datamodel.nexus_writer import to_nexus
import nexusformat.nexus.tree as nx
sjson = parsed_json["substance"][0]
s = Study(**sjson)
print(s)
ne = s.to_nexus(nx.NXroot())
print(ne.tree)

In [ ]:
from  pynanomapper.datamodel.ambit import ProtocolApplication


pa = ProtocolApplication(**json_data)
pa

In [ ]:
from  pynanomapper.datamodel.nexus_writer import to_nexus
import nexusformat.nexus.tree as nx
ne = pa.to_nexus(nx.NXroot())
print(ne.tree)